In [0]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
from keras.preprocessing.sequence import pad_sequences
numpy.random.seed(7)

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals


In [24]:
!git clone --branch r1.13.0 --depth 1 https://github.com/tensorflow/models


fatal: destination path 'models' already exists and is not an empty directory.


In [25]:
!tf_upgrade_v2 -h


usage: tf_upgrade_v2 [-h] [--infile INPUT_FILE] [--outfile OUTPUT_FILE]
                     [--intree INPUT_TREE] [--outtree OUTPUT_TREE]
                     [--copyotherfiles COPY_OTHER_FILES] [--inplace]
                     [--reportfile REPORT_FILENAME] [--mode {DEFAULT,SAFETY}]
                     [--print_all]

Convert a TensorFlow Python file from 1.x to 2.0

Simple usage:
  tf_upgrade_v2.py --infile foo.py --outfile bar.py
  tf_upgrade_v2.py --infile foo.ipynb --outfile bar.ipynb
  tf_upgrade_v2.py --intree ~/code/old --outtree ~/code/new

optional arguments:
  -h, --help            show this help message and exit
  --infile INPUT_FILE   If converting a single file, the name of the file to
                        convert
  --outfile OUTPUT_FILE
                        If converting a single file, the output filename.
  --intree INPUT_TREE   If converting a whole tree of files, the directory to
                        read from (relative or absolute).
  --outtree OUTPUT_TREE


In [0]:
#dataset for alphabets
letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

#mapping
char_to_int = dict((c,i) for i,c in enumerate(letters))
int_to_char = dict((i,c) for i,c in enumerate(letters))


In [28]:
#create input and output pair to train neural network
num_samples = 1000
max_len = 5
dataX = []
datay = []

for i in range(num_samples):
  start = numpy.random.randint(len(letters)-2)
  end = numpy.random.randint(start, min(start+max_len,len(letters)-1))
  input = letters[start:end+1]
  output = letters[end+1]
  dataX.append([char_to_int[char] for char in input])
  datay.append(char_to_int[output])
  print(input, " --> ", output) 

PQRST  -->  U
W  -->  X
O  -->  P
OPQ  -->  R
IJKLM  -->  N
QRSTU  -->  V
ABCD  -->  E
X  -->  Y
GHIJ  -->  K
M  -->  N
XY  -->  Z
QRST  -->  U
ABC  -->  D
JKLMN  -->  O
OP  -->  Q
XY  -->  Z
D  -->  E
T  -->  U
B  -->  C
QRSTU  -->  V
HIJ  -->  K
JKLM  -->  N
ABCDE  -->  F
X  -->  Y
V  -->  W
DE  -->  F
DEFG  -->  H
BCDE  -->  F
EFGH  -->  I
BCDE  -->  F
FG  -->  H
RST  -->  U
TUV  -->  W
STUV  -->  W
LMN  -->  O
P  -->  Q
MNOP  -->  Q
JK  -->  L
MNOP  -->  Q
OPQRS  -->  T
UVWXY  -->  Z
PQRS  -->  T
D  -->  E
EFGH  -->  I
IJK  -->  L
WX  -->  Y
STUV  -->  W
MNOPQ  -->  R
P  -->  Q
WXY  -->  Z
VWX  -->  Y
V  -->  W
HI  -->  J
KLMNO  -->  P
UV  -->  W
JKL  -->  M
ABCDE  -->  F
WXY  -->  Z
M  -->  N
CDEF  -->  G
KLMNO  -->  P
RST  -->  U
RS  -->  T
W  -->  X
J  -->  K
WX  -->  Y
JKLMN  -->  O
MN  -->  O
L  -->  M
BCDE  -->  F
TU  -->  V
MNOPQ  -->  R
NOPQR  -->  S
HIJ  -->  K
JKLM  -->  N
STUVW  -->  X
QRST  -->  U
N  -->  O
VWXY  -->  Z
B  -->  C
UVWX  -->  Y
OP  -->  Q
K  -->  L
C  -->

In [0]:
#as input sequence length varies hence it requires padding
X = pad_sequences(dataX, maxlen = max_len, dtype='float32')

In [0]:
#LSTM requires format [samples, time_steps, features]
X = numpy.reshape(X, (X.shape[0], max_len, 1))

In [0]:
#normalization
X = X/float(len(letters))

In [0]:
#one hot encoding 
y = np_utils.to_categorical(datay)

In [0]:
#one-char to one-char mapping
#stateful : it restes state of network at end of epoch

batch_size = 1
model = Sequential()
model.add(LSTM(32, batch_input_shape=(batch_size,X.shape[1], 1), stateful = True))
model.add(Dense(y.shape[1], activation= 'softmax' ))
model.compile(loss= 'categorical_crossentropy' , optimizer= 'adam' , metrics=[ 'accuracy' ])


In [34]:
from keras.callbacks import ModelCheckpoint

filepath = 'weights.best.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'val_acc', verbose = 1, save_best_only = True, mode = 'max')
callbacks_list = [checkpoint]

for i in range(100):
  model.fit(X, y, nb_epoch=1, batch_size=batch_size, verbose=2, shuffle=False, callbacks = callbacks_list)
  model.reset_states()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Epoch 1/1
 - 10s - loss: 3.1422 - acc: 0.0620


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


Epoch 1/1
 - 9s - loss: 2.8998 - acc: 0.1010
Epoch 1/1
 - 10s - loss: 2.6851 - acc: 0.1340
Epoch 1/1
 - 10s - loss: 2.5216 - acc: 0.1920
Epoch 1/1
 - 10s - loss: 2.3765 - acc: 0.2380
Epoch 1/1
 - 10s - loss: 2.2519 - acc: 0.2530
Epoch 1/1
 - 10s - loss: 2.1556 - acc: 0.2720
Epoch 1/1
 - 9s - loss: 2.0759 - acc: 0.2930
Epoch 1/1
 - 10s - loss: 2.0037 - acc: 0.3240
Epoch 1/1
 - 10s - loss: 1.9371 - acc: 0.3430
Epoch 1/1
 - 10s - loss: 1.8746 - acc: 0.3520
Epoch 1/1
 - 10s - loss: 1.8183 - acc: 0.3620
Epoch 1/1
 - 10s - loss: 1.7677 - acc: 0.3750
Epoch 1/1
 - 10s - loss: 1.7194 - acc: 0.3970
Epoch 1/1
 - 10s - loss: 1.6735 - acc: 0.4180
Epoch 1/1
 - 10s - loss: 1.6323 - acc: 0.4310
Epoch 1/1
 - 10s - loss: 1.5937 - acc: 0.4500
Epoch 1/1
 - 10s - loss: 1.5541 - acc: 0.4540
Epoch 1/1
 - 10s - loss: 1.5179 - acc: 0.4660
Epoch 1/1
 - 10s - loss: 1.4816 - acc: 0.4700
Epoch 1/1
 - 10s - loss: 1.4506 - acc: 0.4770
Epoch 1/1
 - 10s - loss: 1.4225 - acc: 0.4810
Epoch 1/1
 - 10s - loss: 1.3918 - ac

In [35]:
acc = model.evaluate(X,y, batch_size = batch_size)
print("Accuracy: ", acc[1]*100)

1000/1000 [==============================] - 5s 5ms/step
Accuracy:  87.6


In [38]:
model.reset_states()

#testing for random sequences

for i in range(20):
  index = numpy.random.randint(len(dataX))
  pattern = dataX[index]
  x = pad_sequences([pattern], maxlen= max_len, dtype = 'float32')
  x = numpy.reshape(x, (1, max_len, 1))
  x = x/float(len(letters))
  output = model.predict(x)
  output_index = numpy.argmax(output)
  result = int_to_char[output_index]
  input = [int_to_char[val] for val in pattern]
  print(input," --> ", result)

['U', 'V', 'W', 'X', 'Y']  -->  Z
['R', 'S', 'T', 'U', 'V']  -->  W
['K', 'L', 'M']  -->  N
['T', 'U', 'V', 'W', 'X']  -->  Z
['G', 'H', 'I', 'J']  -->  K
['X']  -->  X
['L']  -->  N
['D', 'E', 'F']  -->  G
['Q']  -->  R
['O', 'P', 'Q', 'R']  -->  S
['M', 'N', 'O', 'P', 'Q']  -->  R
['U', 'V']  -->  W
['N', 'O', 'P', 'Q']  -->  R
['L', 'M']  -->  N
['V', 'W', 'X', 'Y']  -->  Z
['E', 'F']  -->  G
['F', 'G', 'H']  -->  I
['R', 'S']  -->  S
['O', 'P', 'Q', 'R']  -->  S
['H', 'I', 'J', 'K', 'L']  -->  M
